In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions
#from modules.models import MyConv2DModel_v0B_Small_CBAM_1DConv_Share
from modules.models import DenseNetModelLarge
from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)

PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 10
NUM_EVAL = 1
CONTINUAL = True

def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

ModelCatalog.register_custom_model('my_model', DenseNetModelLarge)

# config = {"env": MyEnv,
#           "num_workers": NUM_WORKERS,
#           "num_gpus": NUM_GPUS,
#           "num_cpus_per_worker": NUM_CPUS_PER_WORKER,
#           "num_sgd_iter": NUM_SGD_ITER,
#           "lr": LEARNING_RATE,
#           "gamma": GAMMA,  # default=0.99
#           "model": {"custom_model": "my_model"}
#           # "framework": framework
#           }  # use tensorflow 2
eval_env = MyEnv({})
policies = {
    #"blue_1": PolicySpec(config={"gamma": 0.99}),
    #"blue_2": PolicySpec(config={"gamma": 0.95}),
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
}
policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env

config = {"env": MyEnv,"num_gpus": 0,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 0,
          "create_env_on_driver": True,"train_batch_size": 600*5,"batch_mode": "complete_episodes",
          "shuffle_sequences": True, "gamma":0.9995, "lr": 1e-6,
          "clip_actions":True,"normalize_actions":False,"preprocessor_pref": "rllib",
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "explore":True,
          "sgd_minibatch_size": 100, "num_sgd_iter":20,
          "exploration_config": {"type": "StochasticSampling","random_timesteps":600000}, #PPO デフォルト
          #"exploration_config": {"type": "Random"},
          "multiagent": {"policies": policies,  "policy_mapping_fn": policy_mapping_fn}
         }
res_name = "sgd"+str(config["sgd_minibatch_size"])+"sgd_num"+str(config["num_sgd_iter"])+"lr"+str(config["lr"])+"gamma"+str(config["gamma"])
res_name = "test"

conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    trainer.restore(checkpoint_path=model_path)

# Instanciate the evaluation env
eval_env = MyEnv({})
record_mode = 0

C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
2021-12-22 20:41:30,244	INFO trainer.py:723 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also want to then set `eager_tracing=True` in order to reach similar execution speed as with static-graph mode.
2021-12-22 20:41:30,245	INFO ppo.py:167 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2021-12-22 20:41:30,245	INFO trainer.py:745 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or u

In [2]:
for i in range(NUM_EVAL):
    # print(f'\nEvaluation {i}:')
    obs = eval_env.reset()
    done = False

    step_num = 0
    fig = plt.figure(1,figsize=(8.0, 6.0))
    ESC = 0x1B          # ESCキーの仮想キーコード
    trajectory_length = 100
    env_blue_pos = [0]
    env_red_pos = [0]
    env_mrm_pos = [0]
    if record_mode == 0:
        file_name = "test_num" +str(i)
        video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

    while True:
        action_dict = {}
        for j in range(eval_env.blue_num):
            if not eval_env.blue[j].hitpoint == 0:
                #action_dict['blue_' + str(j)] = trainer.compute_action(obs['blue_' + str(j)])
                action_dict['blue_' + str(j)] = trainer.compute_action(obs['blue_' + str(j)],policy_id='blue_' + str(j),
                                                                   clip_actions=True,explore=False)
   
        obs, rewards, dones, infos = eval_env.step(action_dict)
        #for j in range(eval_env.blue_num): 

           # print(action_dict['blue_' + str(j)]["vector_psi_y"],action_dict['blue_' + str(j)]["vector_psi_x"],
           #       action_dict['blue_' + str(j)]["vector_gam_y"],action_dict['blue_' + str(j)]["vector_gam_x"],
           #       action_dict['blue_' + str(j)]["velocity"])
            
            #print('blue_' + str(j),eval_env.blue[j].V, np.rad2deg(eval_env.blue[j].psi),  np.rad2deg(eval_env.blue[j].gam),
            #     eval_env.blue[j].V_ref,np.rad2deg(eval_env.blue[j].psi_ref),  np.rad2deg(eval_env.blue[j].gam_ref))
        env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp= render_env.copy_from_env(eval_env)
        env_blue_pos.append(env_blue_pos_temp)
        env_red_pos.append(env_red_pos_temp)
        env_mrm_pos.append(env_mrm_pos_temp)
        if step_num == 0:
            del env_blue_pos[0]
            del env_red_pos[0]
            del env_mrm_pos[0]

        hist_blue_pos = np.vstack(env_blue_pos)
        hist_red_pos = np.vstack(env_red_pos)
        hist_mrm_pos = np.vstack(env_mrm_pos)
        plt.clf()
        render_env.rend_3d(eval_env,hist_blue_pos,"b",1)
        render_env.rend_3d(eval_env,hist_red_pos,"r",1)
        render_env.rend_3d(eval_env,hist_mrm_pos,"k",1)
        plt.pause(.01)
        step_num = step_num + 1

        done = dones["__all__"]
        if record_mode == 0:
            img = np.array(fig.canvas.renderer.buffer_rgba())
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
            # cv2.imshow('test', img)
            # cv2.waitKey(1)
            # cv2.destroyAllWindows()
            video.write(img.astype('uint8'))
        elif record_mode == 1:
            env.render()
        elif record_mode == 2:
            True #将来のため処理予約

        #env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp = render_env.copy_from_env(eval_env)

        #env_blue_pos.append(env_blue_pos_temp)
        #env_red_pos.append(env_red_pos_temp)
        #env_mrm_pos.append(env_mrm_pos_temp)
        #step_num = step_num + 1
        # エピソードの終了処理
        if dones['__all__']:
            # print(f'all done at {env.steps}')
            if record_mode == 0:
                video.release()
            break

    #del env_blue_pos[0]
    #del env_red_pos[0]
    #del env_mrm_pos[0]

    #hist_blue_pos = np.vstack(env_blue_pos)
    #hist_red_pos = np.vstack(env_red_pos)
    #hist_mrm_pos = np.vstack(env_mrm_pos)

    #f = open(results_file,'wb')
    #f = open("log.pkl","wb")
    #pickle.dump(emv_blue_pos,f)
    #pickle.dump(emv_blue_pos,f)
    #pickle.dump(emv_red_pos,f)
    #pickle.dump(emv_mrm_pos,f)
    #f.close()

    if record_mode == 0:
        video.release()

ray.shutdown()

2021-12-22 20:41:37,615	WARNING deprecation.py:46 -- DeprecationWarning: `compute_action` has been deprecated. Use `compute_single_action` instead. This will raise an error in the future!
2021-12-22 20:41:37,616	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:37,698	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:38,062	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:38,064	WARNING deprecation.py:46 -- Deprecati

2021-12-22 20:41:40,812	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:40,950	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:40,952	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:41,187	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!


2021-12-22 20:41:43,310	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:43,312	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:43,437	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:43,442	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!


2021-12-22 20:41:45,946	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:46,101	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:46,103	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:46,267	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!


2021-12-22 20:41:48,672	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:48,674	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:48,832	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:48,835	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!


2021-12-22 20:41:51,396	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:51,549	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:51,552	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:51,703	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!


2021-12-22 20:41:54,035	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:54,038	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:54,199	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2021-12-22 20:41:54,203	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!


blue_0DOWN
DOWN LOSE
blue_0 True True 114 -100.00529502864272 -101.78467159871074
blue_1 True True 114 -0.0016061293954750201 -0.8249658576786137


C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\ipykernel\eventloops.py:256: RuntimeWarning: coroutine 'Kernel.do_one_iteration' was never awaited
  self.func()
